In [44]:
!pip install psycopg2
!pip install pprint

You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [45]:
import os
import logging

import psycopg2
import psycopg2.extensions
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, Float, MetaData, String
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

In [46]:
!pip install pymongo

You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [47]:
from pymongo import MongoClient

In [48]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [49]:
logger.info("Создаём подключёние к Postgres")
params = {
    'host': '0.0.0.0',
    'port': '5433',
    'user': 'postgres'
}
conn = psycopg2.connect(**params)


2018-08-21 18:53:37,003 : INFO : Создаём подключёние к Postgres


In [50]:
psycopg2.extensions.register_type(
    psycopg2.extensions.UNICODE,
    conn
)
conn.set_isolation_level(
    psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT
)
cursor = conn.cursor()

In [55]:
sql_str = ("""select ratings.movieid as movieid, count(ratings.rating) as ratings_num, avg(ratings.rating) as ratings_avg into movies_2 from ratings group by movieid """)

In [56]:
cursor.execute(sql_str)
conn.commit()

In [57]:
cursor.execute("SELECT * FROM movies_up LIMIT 10")
logger.info(
    "Выгружаем данные из таблицы movies_up: (movieId, ratings_num, ratings_avg)\n{}".format(
        [i for i in cursor.fetchall()])
)


2018-08-21 18:54:13,078 : INFO : Выгружаем данные из таблицы movies_up: (movieId, ratings_num, ratings_avg)
[(6114, 1, 3.5), (273, 27, 3.12962962962963), (3936, 6, 3.0), (128606, 1, 4.5), (4326, 11, 4.09090909090909), (2520, 15, 3.36666666666667), (25886, 1, 4.0), (1750, 1, 2.0), (4321, 28, 3.46428571428571), (5230, 1, 4.0)]


In [58]:
cursor.execute("SELECT * FROM movies_top LIMIT 10")
logger.info(
    "Выгружаем данные из таблицы movies_top: (movieId, ratings_num, ratings_avg)\n{}".format(
        [i for i in cursor.fetchall()])
)


2018-08-21 18:54:15,347 : INFO : Выгружаем данные из таблицы movies_top: (movieId, ratings_num, ratings_avg)
[(6114, 1, 3.5), (273, 27, 3.12962962962963), (3936, 6, 3.0), (128606, 1, 4.5), (4326, 11, 4.09090909090909), (2520, 15, 3.36666666666667), (25886, 1, 4.0), (1750, 1, 2.0), (4321, 28, 3.46428571428571), (5230, 1, 4.0)]


Задание по SQLAlchemy

In [60]:
Base = declarative_base()

In [61]:
class MoviesTop(Base):
    __tablename__ = 'movies_top'
    movieid = Column(Integer, primary_key=True)
    ratings_num = Column(Float)
    ratings_avg = Column(Float)
    def __repr__(self):
        return "<User(movieid='%s', ratings_num='%s', ratings_avg='%s')>" % (self.movieid, self.ratings_num, self.ratings_avg)

In [123]:
engine = create_engine('postgresql://postgres:@{}:{}'.format('0.0.0.0', 5433))
Session = sessionmaker(bind=engine)
session = Session()

In [91]:
for m in session.query(MoviesTop).order_by(MoviesTop.ratings_num)[1:3]:
    print(m)

<User(movieid='128606', ratings_num='1.0', ratings_avg='4.5')>
<User(movieid='25886', ratings_num='1.0', ratings_avg='4.0')>


In [92]:
test = session.query(MoviesTop).order_by(MoviesTop.ratings_num)[1:3]

In [93]:
test

[<User(movieid='128606', ratings_num='1.0', ratings_avg='4.5')>,
 <User(movieid='25886', ratings_num='1.0', ratings_avg='4.0')>]

In [112]:
# выберите контент у которого больше 15 оценок (используйте filter)

In [111]:
for i in session.query(MoviesTop.movieid, MoviesTop.ratings_num).group_by(MoviesTop.movieid, MoviesTop.ratings_num).filter(MoviesTop.ratings_num > 15).all():
    print(i)

(348, 40.0)
(3986, 20.0)
(104, 94.0)
(44195, 40.0)
(1388, 27.0)
(1527, 118.0)
(276, 22.0)
(1645, 60.0)
(2706, 111.0)
(3717, 56.0)
(2664, 19.0)
(34048, 40.0)
(410, 73.0)
(60072, 18.0)
(380, 198.0)
(58559, 121.0)
(371, 20.0)
(880, 25.0)
(89492, 16.0)
(1517, 107.0)
(53125, 33.0)
(5528, 19.0)
(176, 18.0)
(3257, 29.0)
(785, 60.0)
(1022, 55.0)
(3175, 65.0)
(140, 23.0)
(493, 23.0)
(2058, 33.0)
(994, 22.0)
(265, 62.0)
(434, 106.0)
(3157, 27.0)
(949, 16.0)
(7099, 20.0)
(3916, 39.0)
(1220, 94.0)
(1955, 31.0)
(2140, 28.0)
(1029, 42.0)
(3686, 20.0)
(2006, 55.0)
(1233, 51.0)
(41285, 16.0)
(3993, 16.0)
(431, 32.0)
(1907, 42.0)
(2053, 17.0)
(647, 31.0)
(2403, 18.0)
(6296, 21.0)
(450, 16.0)
(750, 105.0)
(471, 49.0)
(6957, 19.0)
(122, 19.0)
(2863, 30.0)
(2881, 43.0)
(109374, 35.0)
(7325, 17.0)
(2997, 126.0)
(21, 95.0)
(3451, 18.0)
(2301, 18.0)
(69, 22.0)
(5219, 22.0)
(3910, 23.0)
(910, 66.0)
(273, 27.0)
(2046, 17.0)
(1287, 46.0)
(524, 36.0)
(23, 22.0)
(1211, 19.0)
(1792, 22.0)
(1057, 17.0)
(374, 20.0)


In [116]:
# средний рейтинг больше 3.5 

In [113]:
for i in session.query(MoviesTop.movieid, MoviesTop.ratings_avg).group_by(MoviesTop.movieid, MoviesTop.ratings_avg).filter(MoviesTop.ratings_avg > 3.5).all():
    print(i)

(87222, 3.83333333333333)
(26524, 4.5)
(118105, 4.0)
(85342, 4.1)
(1408, 3.61627906976744)
(104419, 4.5)
(3700, 3.66666666666667)
(48268, 4.0)
(5893, 3.66666666666667)
(7935, 4.0)
(299, 3.75)
(73759, 4.0)
(3454, 4.0)
(33669, 3.75)
(3342, 4.0)
(1957, 3.94047619047619)
(39231, 3.75)
(4186, 3.6)
(74677, 4.0)
(93265, 4.5)
(26251, 4.0)
(6337, 3.83333333333333)
(31682, 4.5)
(8873, 3.67307692307692)
(117192, 4.75)
(1846, 4.5)
(1411, 4.16)
(1860, 4.66666666666667)
(2132, 4.19565217391304)
(900, 3.95454545454545)
(534, 4.2)
(1680, 3.69230769230769)
(308, 4.15909090909091)
(2231, 3.84722222222222)
(2859, 4.0)
(95873, 3.83333333333333)
(100556, 4.125)
(6829, 4.0)
(4705, 3.75)
(26258, 4.0)
(5818, 4.0)
(40732, 3.85714285714286)
(6790, 3.625)
(3200, 4.3)
(96588, 3.92857142857143)
(38304, 4.16666666666667)
(96832, 5.0)
(84116, 4.0)
(26737, 4.0)
(875, 4.0)
(84847, 4.0)
(2131, 4.5)
(156609, 3.75)
(6869, 4.125)
(64620, 3.85)
(111921, 3.91666666666667)
(3265, 4.1)
(6611, 4.5)
(69529, 4.0)
(1283, 4.119047

In [117]:
# отсортированный по среднему рейтингу

In [124]:
for i in session.query(MoviesTop.movieid, MoviesTop.ratings_avg).group_by(MoviesTop.movieid, MoviesTop.ratings_avg).order_by(-MoviesTop.ratings_avg).all():
    print(i)

(60384, 5.0)
(130522, 5.0)
(163949, 5.0)
(4252, 5.0)
(26797, 5.0)
(3165, 5.0)
(3656, 5.0)
(142068, 5.0)
(2945, 5.0)
(4565, 5.0)
(74089, 5.0)
(65037, 5.0)
(101962, 5.0)
(91690, 5.0)
(4789, 5.0)
(4428, 5.0)
(2981, 5.0)
(93324, 5.0)
(5765, 5.0)
(27724, 5.0)
(8208, 5.0)
(2649, 5.0)
(764, 5.0)
(110873, 5.0)
(54328, 5.0)
(961, 5.0)
(26599, 5.0)
(5960, 5.0)
(5244, 5.0)
(26749, 5.0)
(7302, 5.0)
(26791, 5.0)
(77435, 5.0)
(32525, 5.0)
(61250, 5.0)
(96832, 5.0)
(4140, 5.0)
(3575, 5.0)
(6342, 5.0)
(141886, 5.0)
(5473, 5.0)
(100553, 5.0)
(78903, 5.0)
(3746, 5.0)
(43267, 5.0)
(55555, 5.0)
(6369, 5.0)
(2650, 5.0)
(112577, 5.0)
(7260, 5.0)
(41573, 5.0)
(3909, 5.0)
(136447, 5.0)
(136445, 5.0)
(4584, 5.0)
(1543, 5.0)
(107447, 5.0)
(26843, 5.0)
(4632, 5.0)
(32460, 5.0)
(141124, 5.0)
(51471, 5.0)
(8123, 5.0)
(2647, 5.0)
(6769, 5.0)
(140739, 5.0)
(107559, 5.0)
(7932, 5.0)
(59392, 5.0)
(50703, 5.0)
(2982, 5.0)
(97057, 5.0)
(5062, 5.0)
(4617, 5.0)
(3851, 5.0)
(107978, 5.0)
(6033, 5.0)
(64278, 5.0)
(4201, 5.0

In [118]:
# контент со всеми предыдущими параметрами сохраняем в массив top_rated_content_ids

In [128]:
# выгрузим данные полностью для проверки

In [126]:
for i in session.query(MoviesTop.movieid, MoviesTop.ratings_num, MoviesTop.ratings_avg).group_by(MoviesTop.movieid, MoviesTop.ratings_num, MoviesTop.ratings_avg).filter(MoviesTop.ratings_num > 15, MoviesTop.ratings_avg > 3.5).order_by(-MoviesTop.ratings_avg).all():
    print(i)

(858, 200.0, 4.4875)
(318, 311.0, 4.48713826366559)
(1945, 29.0, 4.44827586206897)
(1147, 16.0, 4.4375)
(926, 38.0, 4.43421052631579)
(1217, 26.0, 4.42307692307692)
(969, 50.0, 4.42)
(3035, 17.0, 4.41176470588235)
(2064, 42.0, 4.39285714285714)
(913, 62.0, 4.38709677419355)
(7502, 22.0, 4.38636363636364)
(1221, 135.0, 4.38518518518519)
(905, 25.0, 4.38)
(50, 201.0, 4.37064676616915)
(3462, 32.0, 4.359375)
(898, 37.0, 4.35135135135135)
(1228, 50.0, 4.35)
(1252, 76.0, 4.33552631578947)
(2318, 23.0, 4.32608695652174)
(904, 92.0, 4.31521739130435)
(3730, 23.0, 4.30434782608696)
(1203, 74.0, 4.30405405405405)
(527, 244.0, 4.30327868852459)
(1254, 30.0, 4.3)
(6016, 69.0, 4.29710144927536)
(922, 39.0, 4.29487179487179)
(3683, 24.0, 4.29166666666667)
(194, 24.0, 4.29166666666667)
(1237, 16.0, 4.28125)
(2019, 54.0, 4.27777777777778)
(1276, 46.0, 4.27173913043478)
(1299, 35.0, 4.27142857142857)
(908, 87.0, 4.27011494252874)
(3088, 19.0, 4.26315789473684)
(1260, 21.0, 4.26190476190476)
(1172, 46.

In [129]:
#сделаем массив только с id

In [296]:
top_rated_content_ids = []
for i in session.query(MoviesTop.movieid).group_by(MoviesTop.movieid, MoviesTop.ratings_num, MoviesTop.ratings_avg).filter(MoviesTop.ratings_num > 15, MoviesTop.ratings_avg > 3.5).order_by(-MoviesTop.ratings_avg).all():
    top_rated_content_ids.append(i[0])

In [434]:
print(top_rated_content_ids[:5])

[858, 318, 1945, 1147, 926]


In [288]:
top_rated_query = session.query(MoviesTop)

In [254]:
logger.info("Выборка из top_rated_query\n{}".format([i for i in top_rated_query.limit(4)]))

2018-08-21 22:02:48,217 : INFO : Выборка из top_rated_query
[<User(movieid='6114', ratings_num='1.0', ratings_avg='3.5')>, <User(movieid='273', ratings_num='27.0', ratings_avg='3.12962962962963')>, <User(movieid='3936', ratings_num='6.0', ratings_avg='3.0')>, <User(movieid='128606', ratings_num='1.0', ratings_avg='4.5')>]


Задание по PyMongo

In [202]:
mongo = MongoClient(**{
    'host': '0.0.0.0',
    'port': int(27017)
})

In [233]:
db = mongo["movies"]
tags_collection = db['tags']

In [234]:
tags_collection

Collection(Database(MongoClient(host=['0.0.0.0:27017'], document_class=dict, tz_aware=False, connect=True), 'movies'), 'tags')

In [235]:
print('Count',tags_collection.count())

Count 158680


/Users/staroverova/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  if __name__ == '__main__':


In [236]:
for i in tags_collection.find().limit(5):
    print(i)

{'_id': ObjectId('5b7c5ce247729f69a8a161c1'), 'id': 931, 'name': 'jealousy'}
{'_id': ObjectId('5b7c5ce247729f69a8a161c2'), 'id': 4290, 'name': 'toy'}
{'_id': ObjectId('5b7c5ce247729f69a8a161c3'), 'id': 5202, 'name': 'boy'}
{'_id': ObjectId('5b7c5ce247729f69a8a161c4'), 'id': 6054, 'name': 'friendship'}
{'_id': ObjectId('5b7c5ce247729f69a8a161c5'), 'id': 9713, 'name': 'friends'}


In [304]:
mongo_query = tags_collection.find({'id': {'$in': top_rated_content_ids}})

In [305]:
mongo_docs = [i for i in mongo_query]

In [306]:
print("Достали документы из Mongo: {}".format(mongo_docs[:5]))

Достали документы из Mongo: [{'_id': ObjectId('5b7c5ce247729f69a8a161c1'), 'id': 931, 'name': 'jealousy'}, {'_id': ObjectId('5b7c5ce247729f69a8a161dc'), 'id': 4995, 'name': 'aging'}, {'_id': ObjectId('5b7c5ce247729f69a8a161fc'), 'id': 380, 'name': 'brother brother relationship'}, {'_id': ObjectId('5b7c5ce247729f69a8a16201'), 'id': 949, 'name': 'terrorist'}, {'_id': ObjectId('5b7c5ce247729f69a8a16203'), 'id': 1653, 'name': 'explosive'}]


In [307]:
id_tags = [(i['id'], i['name']) for i in mongo_docs]

Задание по Pandas

In [421]:
tags_df = pd.DataFrame(id_tags, columns=['movieid', 'tags'])

In [ ]:
#совсем простой вариант, просто считаем значения по колонке - вряд ли в каждом фильме будет два раза один и тот же тег

In [425]:
tags_df['tags'].value_counts().head(5)

female nudity                   563
world war ii                    394
monster                         285
jealousy                        197
brother brother relationship    176
Name: tags, dtype: int64

In [427]:
#более сложный и правильный вариант - через groupby

In [428]:
tags_df.head()

,movieid,tags
0,931,jealousy
1,4995,aging
2,380,brother brother relationship
3,949,terrorist
4,1653,explosive


In [429]:
coun_tags = tags_df.groupby(by = 'tags', as_index=False).count() #групиируем по тегам и выводим счетчки

In [430]:
top_tags = coun_tags.rename(columns = {'movieid': 'count_tags'}) #заменяем название, чтобы было понятно

In [431]:
top_5_tags = top_tags.sort_values(by='count_tags',axis=0, ascending=False).head(5)

In [432]:
print(top_5_tags)

                             tags  count_tags
99                  female nudity         563
347                  world war ii         394
200                       monster         285
149                      jealousy         197
34   brother brother relationship         176


In [433]:
logger.info("Домашка выполнена!")

2018-08-22 12:00:56,657 : INFO : Домашка выполнена!
